In [1]:
import weakref

# Creating a cache
We simulate the creation of a complex object that we want to store in a cache for as long as we need it.

### Basic cache

In [4]:
class ComplexObject:

    _cache = {}
    
    def __new__(self, _id):
        
        obj = ComplexObject._cache.get(_id, None)
        
        if obj:
            print(f"{obj} found in cache!")
        else:
            obj = super().__new__(self)
            obj._id = _id
            print(f"{obj} not found in cache, created!")
            self._cache[_id] = obj
        
        return self._cache[_id]
    
    def __repr__(self):
        return f"{type(self).__name__}(_id={self._id})"

    def __del__(self):
        print(f"{self} deleted")

In [5]:
ComplexObject._cache

{}

In [6]:
o1 = ComplexObject(1)

ComplexObject(_id=1) not found in cache, created!


In [7]:
ComplexObject._cache

{1: ComplexObject(_id=1)}

In [8]:
_o1 = ComplexObject(1)

ComplexObject(_id=1) found in cache!


In [9]:
o1 is _o1

True

In [10]:
del o1 # object is not deleted because there is stiil a reference in the cache

In [11]:
del _o1

In [12]:
ComplexObject._cache

{1: ComplexObject(_id=1)}

The object is not deleted because a reference still exist in the cache.

### Cache with weak references
In this example, only a weak ref of the object is stored in the dictionary. But when instantiated, hard ref are returned

In [15]:
class WRComplexObject:

    _cache = {}
    
    def __new__(self, _id):
        
        obj = WRComplexObject._cache.get(_id, None)

        # instantiate the object and store a weak ref in the dictionary
        # return a hard ref to the object
        if not obj:
            
            obj = super().__new__(self)
            obj._id = _id
            print(f"{obj} not found in cache, created!")
            
            self._cache[_id] = weakref.ref(obj)
            return obj
        
        # access object through its weak ref
        # return a hard ref to the object through weakref.ref()
        else:
            print(f"{obj()} found in cache!")
            obj = self._cache[_id]
            return self._cache[_id]()
    
    def __repr__(self):
        return f"{type(self).__name__}(_id={self._id})"

    def __del__(self):
        print(f"{self} deleted")
        # print(f"clear cache")
        # del WRComplexObject[self._id]

In [16]:
WRComplexObject._cache

{}

In [17]:
w1 = WRComplexObject(1)

WRComplexObject(_id=1) not found in cache, created!


In [18]:
WRComplexObject._cache

{1: <weakref at 0x00000239E5445260; to 'WRComplexObject' at 0x00000239E4F50080>}

In [19]:
_w1 = WRComplexObject(1)

WRComplexObject(_id=1) found in cache!


In [36]:
w1 is _w1

True

In [40]:
del w1

In [42]:
del _w1

WRComplexObject(_id=1) deleted


In [44]:
WRComplexObject._cache

{1: <weakref at 0x00000239E5445260; dead>}

Now we see, that when deleting all references to object the cache is cleared. This avoid storing unused object in the memory space.